# Variational Quantum Error Correction with QVECTOR

In this notebook, we will optimize a Quantum Error Correction (QEC) problem. 
Here, our set of states $\{\psi_i\}_{i=1}^N$ are sampled from the Haar distribution (or an approximation thereof) and we optimize the cost 
$$L(\vec{p}, \vec{q}) = \sum_{\vert \psi_i \rangle \in \mathcal{S}} \frac{1}{\vert\mathcal{S}\vert} E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$$
where $\vec{p}, \vec{q}$ are parameters of a variational circuit, and $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q}) = \langle \psi_i \vert \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{N} \mathcal{V}(\vec{p}) \vert \psi_i \rangle$, with $\mathcal{V}$ the encoding operator, $\mathcal{N}$ the noise model, and $\mathcal{W}$ the decoding operator. The noise model here will be a bitflip channel (with probability $0.1$) acting on all qubits.

In [1]:
import typing as ty
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

# Initialise the number of qubits that interest us.
k, n, r = 1, 3, 1
nbqbits = n + r

In [2]:
layers = 3

def random_angles(*shape: int) -> np.ndarray:
    return 2 * np.pi * np.random.rand(*shape)

if k == 1:
    # Only consider the 6 one qubit stabilizer states:
    dataset_size = 6
    dataset = [
        [qml.Identity(wires=[0])], [qml.PauliX(wires=[0])],  # |0>, |1>
        [qml.Hadamard(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0])],  # |+>, |->
        [qml.Hadamard(wires=[0]), qml.S(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0]), qml.S(wires=[0])]  # |+i>, |-i>
    ]
    
elif k > 1:
    # Build the dataset here with qml.SimplifiedTwoDesign
    # See https://docs.pennylane.ai/en/stable/code/api/pennylane.SimplifiedTwoDesign.html
    dataset_size = 200
    dataset = [[qml.SimplifiedTwoDesign(initial_layer_weights=random_angles(k), weights=random_angles(layers, k - 1, 2), wires=range(k))] for _ in range(dataset_size)]

In [3]:
# Get the Hamiltonian of interest
coefficients_cost = [- 1.0 / k for _ in range(k)]
projector = np.zeros((2, 2))
projector[0, 0] = 1
vqec_hamiltonian_term = [qml.Hermitian(projector,wires=i) for i in range(k)]
hamiltonian_of_interest = qml.Hamiltonian(coefficients_cost, vqec_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [4]:
def circuit_construction(
    weights: np.ndarray,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    if parameterised_circuit is None:
        parameterised_circuit = qml.StronglyEntanglingLayers
        
    encoding_shape = parameterised_circuit.shape(layers, n)
    decoding_shape = parameterised_circuit.shape(layers, n + r)
    encoding_size = np.prod(encoding_shape)
    decoding_size = np.prod(decoding_shape)
    encoding_weights = weights[:encoding_size].reshape(encoding_shape)
    decoding_weights = weights[encoding_size:encoding_size + decoding_size].reshape(decoding_shape)
    
    # Apply S         to range(k)     to prepare an approximate 2-design state
    for op in data_circuit:
        qml.apply(op)
    
    # Apply V(p)      to range(n)     to encode the state
    parameterised_circuit(encoding_weights, wires=range(n))
    for i in range(n):
        qml.BitFlip(0.1, wires=i)
    # Apply W(q)      to range(n + r) to correct potential errors
    parameterised_circuit(decoding_weights, wires=range(n + r))
    # Apply V^{-1}(p) to range(n)     to decode the state
    qml.adjoint(parameterised_circuit(encoding_weights, wires=range(n)))
    
    # Apply S^{-1}    to range(k)     to un-prepare.
    for op in reversed(data_circuit):
        qml.apply(op.inv())
    
def cost_function(
    weights: np.ndarray,
    hamiltonian_terms: qml.operation.Operator,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    circuit_construction(weights, data_circuit, parameterised_circuit)
    return qml.sample(hamiltonian_terms)

In [5]:
analytic_dev = qml.device("default.mixed", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint, parameterised_circuit = None):
    circuit_construction(weights, dataset[index_datapoint], parameterised_circuit)
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights, parameterised_circuit = None):
    cost = 0.0
    for m in range(dataset_size):
        cost += cost_analytic_one_circuit(weights, m, parameterised_circuit)
    cost = 1.0 + cost / dataset_size
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [6]:
ansatz = qml.StronglyEntanglingLayers

parameter_size: int = np.prod(ansatz.shape(layers, n)) + np.prod(ansatz.shape(layers, n + r))

opt = Refoqus(nbqbits, dataset, vqec_hamiltonian_term, coefficients_cost, param_shape=(parameter_size,), function_cost_term_tosample=cost_function, min_shots=2, device_name="default.mixed")
params = random_angles(parameter_size)
niter = 20

cost_refoqus = [cost_analytic_alldataset(params, ansatz)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params, ansatz))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")

/home/charles/anaconda3/envs/hackenv/lib/python3.10/site-packages/pennylane/operation.py:1506: UserWarning: In-place inversion with inverse is deprecated. Please use qml.adjoint or qml.pow instead.
  warnings.warn(


Step 0: cost = 0.4495655337184422, shots_used = 252
Step 1: cost = 0.4465745861188841, shots_used = 590
Step 2: cost = 0.49308851215111804, shots_used = 1142
Step 3: cost = 0.4612971715359162, shots_used = 1902
Step 4: cost = 0.4421812429003372, shots_used = 2906
Step 5: cost = 0.4342654615347422, shots_used = 3954
Step 6: cost = 0.45427969698165294, shots_used = 5486
Step 7: cost = 0.44793503514792377, shots_used = 7876
Step 8: cost = 0.42454910098941967, shots_used = 11352
Step 9: cost = 0.43304557319586057, shots_used = 18210
Step 10: cost = 0.42829795057682596, shots_used = 26184
Step 11: cost = 0.4155404368675879, shots_used = 39700
Step 12: cost = 0.407405096949191, shots_used = 64894
Step 13: cost = 0.40409094257218403, shots_used = 102724
Step 14: cost = 0.4037855142938185, shots_used = 137992
Step 15: cost = 0.39805755324754444, shots_used = 177082
Step 16: cost = 0.39626363445747925, shots_used = 224786
Step 17: cost = 0.3936877621193865, shots_used = 278700
Step 18: cost = 0